## Немного про межпроцессорную коммуникацию

В multiprocessing в Python есть два основных способа организации обмена данными между процессами — это Queue и Pipe. Хотя оба объекта служат для межпроцессной передачи, у них есть заметные отличия:

In [ ]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

info('main line')
p = Process(target=f, args=('bob',))
p.start()
p.join()

main line
module name: __main__
parent process: 90
process id: 2344
function f
 module name:__main__
parent process: 2344
process id: 21561
hello bob


### Пример 1.1: Использование multiprocessing.Queue

Задача:

	•	Создать процессы, которые принимают сообщения из очереди и выводят их на экран, пока не получат сигнал остановки.
	•	Главный процесс кладёт в очередь несколько сообщений и затем команду на остановку.


In [ ]:
import multiprocessing
import time

def worker(queue):
    """Процесс-потребитель, считывает данные из очереди"""
    while True:
        msg = queue.get()
        if msg == "STOP":
            print(f"[{multiprocessing.current_process().name}] Получил STOP, завершаюсь.")
            break
        print(f"[{multiprocessing.current_process().name}] Получил сообщение: {msg}")
        time.sleep(0.5)

if __name__ == '__main__':
    # Создаём очередь
    msg_queue = multiprocessing.Queue()

    # Создаём два рабочих процесса
    p1 = multiprocessing.Process(target=worker, args=(msg_queue,), name="Worker-1")
    p2 = multiprocessing.Process(target=worker, args=(msg_queue,), name="Worker-2")

    p1.start()
    p2.start()

    # Кладём в очередь несколько сообщений
    for i in range(5):
        msg_queue.put(f"Сообщение #{i}")

    # Команда на остановку для каждого процесса
    msg_queue.put("STOP")
    msg_queue.put("STOP")

    # Ждём завершения процессов
    p1.join()
    p2.join()

    print("Главный процесс завершён.")

[Worker-1] Получил сообщение: Сообщение #0[Worker-2] Получил сообщение: Сообщение #1

[Worker-1] Получил сообщение: Сообщение #2
[Worker-2] Получил сообщение: Сообщение #3
[Worker-1] Получил сообщение: Сообщение #4
[Worker-2] Получил STOP, завершаюсь.
[Worker-1] Получил STOP, завершаюсь.
Главный процесс завершён.


### Пример 1.2: Использование multiprocessing.Pipe

Задача:

	•	Создать два процесса, которые обмениваются данными через двунаправленный канал Pipe.
	•	Один процесс генерирует данные, другой процесс их обрабатывает и отправляет обратно.

In [1]:
# этот код подвисает, хотя сгенерирован о1 pro, как исправить?

import multiprocessing
import time

def producer(conn):
    """Генератор данных"""
    for i in range(5):
        data = f"Данные {i}"
        print(f"[Producer] Отправка: {data}")
        conn.send(data)
        time.sleep(0.5)
    # Сигнал о завершении
    conn.send("STOP")
    conn.close()

def consumer(conn):
    """Обработчик данных"""
    while True:
        data = conn.recv()
        if data == "STOP":
            print("[Consumer] Получен STOP, завершаюсь.")
            conn.send("STOP")
            break

        print(f"[Consumer] Получил: {data}, обрабатываю...")
        time.sleep(0.5)
        # Отправляем ответ обратно
        conn.send(f"Обработано: {data}")

if __name__ == '__main__':
    # Создаём двунаправленный канал
    parent_conn, child_conn = multiprocessing.Pipe(duplex=True)

    # Создаём процессы producer и consumer
    p1 = multiprocessing.Process(target=producer, args=(parent_conn,))
    p2 = multiprocessing.Process(target=consumer, args=(child_conn,))

    p1.start()
    p2.start()

    # Получаем ответы от consumer, пока не встретим STOP
    while True:
        if parent_conn.poll():  # Проверяем, есть ли данные в канале
            response = parent_conn.recv()
            if response == "STOP":
                print("[Main] Получен сигнал STOP. Завершаю работу.")
                break
            print(f"[Main] Ответ от consumer: {response}")
        time.sleep(0.1)

    p1.join()
    p2.join()
    print("[Main] Главный процесс завершён.")

[Producer] Отправка: Данные 0
[Consumer] Получил: Данные 0, обрабатываю...
[Producer] Отправка: Данные 1
[Consumer] Получил: Данные 1, обрабатываю...
[Main] Ответ от consumer: Обработано: Данные 0
[Producer] Отправка: Данные 2
[Consumer] Получил: Данные 2, обрабатываю...
[Main] Ответ от consumer: Обработано: Данные 1
[Producer] Отправка: Данные 3
[Consumer] Получил: Данные 3, обрабатываю...
[Main] Ответ от consumer: Обработано: Данные 2
[Producer] Отправка: Данные 4
[Consumer] Получил: Данные 4, обрабатываю...
[Main] Ответ от consumer: Обработано: Данные 3
[Consumer] Получен STOP, завершаюсь.
[Main] Ответ от consumer: Обработано: Данные 4
[Main] Получен сигнал STOP. Завершаю работу.
[Main] Главный процесс завершён.


### Задача 0: Подключение и работа с SQLite (0б)


1. Создайте базу данных `school.db` и создайте таблицу `students` со следующими полями:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `name` (TEXT),
   - `grade` (INTEGER).
2. Вставьте данные о трёх студентах:
   - `Alice`, оценка: 85;
   - `Bob`, оценка: 90;
   - `Charlie`, оценка: 75.
3. Выведите на экран всех студентов с оценкой выше 80.
4. Обновите оценку студента `Charlie` на 80.
5. Удалите студента `Bob`.
6. Выведите всю информацию из базы данных.


In [2]:
import sqlite3

# Подключение к базе данных
connection = sqlite3.connect("school.db")
cursor = connection.cursor()

# Создание таблицы
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    grade INTEGER NOT NULL
)
""")

# Вставка данных
cursor.executemany("INSERT INTO students (name, grade) VALUES (?, ?)", [
    ("Alice", 85),
    ("Bob", 90),
    ("Charlie", 75)
])
connection.commit()

# Вывод студентов с оценкой выше 80
cursor.execute("SELECT * FROM students WHERE grade > 80")
print("Студенты с оценкой выше 80:")
for row in cursor.fetchall():
    print(row)

# Обновление оценки Charlie
cursor.execute("UPDATE students SET grade = 80 WHERE name = 'Charlie'")
connection.commit()

# Удаление Bob
cursor.execute("DELETE FROM students WHERE name = 'Bob'")
connection.commit()

# Проверка оставшихся студентов
cursor.execute("SELECT * FROM students")
print("Оставшиеся студенты:")
for row in cursor.fetchall():
    print(row)

connection.close()

Студенты с оценкой выше 80:
(1, 'Alice', 85)
(2, 'Bob', 90)
Оставшиеся студенты:
(1, 'Alice', 85)
(3, 'Charlie', 80)


### Задача 2025 -1: Использование оконных функций (1.5б)

1. Создайте SQLite-базу данных `sales.db` с таблицей `sales`:
   - `order_id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `city` (TEXT),
   - `manager` (TEXT),
   - `revenue` (INTEGER).
2. Вставьте данные:
   - `East`, `Sam`, 450;
   - `East`, `Linda`, 350;
   - `West`, `John`, 600;
   - `West`, `Lucy`, 400;
   - `East`, `Lucy`, 250.
3. Напишите SQL-запрос с оконными функциями для:
   - Вычисление Обьема продаж.
   - Вычисления ранга (`rank`)
   - Посчитайте персентиль возможно? (`NTILE`)

**Ожидаемый результат:**
```
('East', 'Sam',   450, 1050, 1)
('East', 'Linda', 350, 1050, 2)
('East', 'Lucas', 250, 1050, 3)
('West', 'John',  600, 1000, 1)
('West', 'Lucy',  400, 1000, 2)
```

In [7]:
'''
SELECT city, manager, revenue,
  SUM(revenue) OVER (PARTITION BY city) AS total_sales,
  RANK() OVER (PARTITION BY city ORDER BY revenue DESC) AS rank,
  NTILE(3) OVER (PARTITION BY city ORDER BY revenue DESC) AS percentile
FROM sales
ORDER BY city, rank;
'''

'SELECT city, manager, revenue,\n  SUM(revenue) OVER (PARTITION BY city) AS total_sales,\n  RANK() OVER (PARTITION BY city ORDER BY revenue DESC) AS rank,\n  NTILE(3) OVER (PARTITION BY city ORDER BY revenue DESC) AS percentile\nFROM sales\nORDER BY city, rank;\n'

In [8]:
import sqlite3
import pandas as pd

connection = sqlite3.connect("sales.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT,
    manager TEXT,
    revenue INTEGER
)
""")

cursor.executemany("""
INSERT INTO sales (city, manager, revenue) VALUES (?, ?, ?)
""", [
    ("East", "Sam", 450),
    ("East", "Linda", 350),
    ("West", "John", 600),
    ("West", "Lucy", 400),
    ("East", "Lucy", 250)
])

connection.commit()

query = """
SELECT city, manager, revenue,
       SUM(revenue) OVER (PARTITION BY city) AS total_sales,
       RANK() OVER (PARTITION BY city ORDER BY revenue DESC) AS rank,
       NTILE(3) OVER (PARTITION BY city ORDER BY revenue DESC) AS percentile
FROM sales
ORDER BY city, rank;
"""

df = pd.read_sql_query(query, connection)

connection.close()

print(df)

   city manager  revenue  total_sales  rank  percentile
0  East     Sam      450         1050     1           1
1  East   Linda      350         1050     2           2
2  East    Lucy      250         1050     3           3
3  West    John      600         1000     1           1
4  West    Lucy      400         1000     2           2


### Задача 3: Использование индексов (0.5 б)

1. Создайте SQLite-базу данных `large_library.db` с таблицей `books`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `title` (TEXT),
   - `author` (TEXT),
   - `year` (INTEGER).
2. Вставьте 1 миллион случайных записей с помощью Python.
3. Создайте индекс на колонке `author`.
4. Напишите запрос для поиска всех книг автора `"Author_100"`. Замерьте время выполнения запроса до и после создания индекса.

In [11]:
import sqlite3
import time
import random

# Подключение к базе данных
connection = sqlite3.connect("large_library.db")
cursor = connection.cursor()

# Создание таблицы
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year INTEGER NOT NULL
)
""")


if not cursor.execute("SELECT COUNT(*) FROM books").fetchone()[0]:
    books = [
        (f"Book_{i}", f"Author_{random.randint(1, 1000)}", random.randint(1900, 2023))
        for i in range(1, 1000001)
    ]
    cursor.executemany("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", books)
    connection.commit()

# Вставка данных
if not cursor.execute("SELECT COUNT(*) FROM books").fetchone()[0]:
    # Ваш код здесь
    cursor.executemany("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", books)
    connection.commit()

# Замер времени до индекса
start = time.time()
cursor.execute("SELECT * FROM books WHERE author = 'Author_100'")
cursor.fetchall()
print(f"Время выполнения без индекса: {time.time() - start:.4f} секунд")

cursor.execute("CREATE INDEX IF NOT EXISTS idx_author ON books(author)")
connection.commit()

# Замер времени после индекса
start = time.time()
cursor.execute("SELECT * FROM books WHERE author = 'Author_100'")
cursor.fetchall()
print(f"Время выполнения с индексом: {time.time() - start:.4f} секунд")

connection.close()

Время выполнения без индекса: 0.1783 секунд
Время выполнения с индексом: 0.0067 секунд


### Задача 4: Использование ограничений (constraints)  (0.5 б)

1. Создайте базу данных `university.db` с таблицей `students`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `name` (TEXT, NOT NULL),
   - `email` (TEXT, UNIQUE),
   - `age` (INTEGER, CHECK(age >= 18)).
2. Попробуйте вставить записи:
   - `Alice`, `alice@example.com`, 20;
   - `Bob`, `bob@example.com`, 17 (должна вызвать ошибку CHECK);
   - `Charlie`, `alice@example.com`, 22 (должна вызвать ошибку UNIQUE).
3. Добавьте индекс на поле `name` для ускорения поиска.
4. Напишите запрос для выборки студентов, чей возраст больше 19.

In [12]:
import sqlite3

connection = sqlite3.connect("university.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    age INTEGER CHECK(age >= 18)
)
""")

students_data = [
    ("Alice", "alice@example.com", 20),
    ("Bob", "bob@example.com", 17),  # Ошибка CHECK
    ("Charlie", "alice@example.com", 22)  # Ошибка UNIQUE
]

for student in students_data:
    try:
        cursor.execute("INSERT INTO students (name, email, age) VALUES (?, ?, ?)", student)
        connection.commit()
    except sqlite3.IntegrityError as e:
        print(f"Ошибка вставки: {e}")

cursor.execute("CREATE INDEX IF NOT EXISTS idx_name ON students(name)")
connection.commit()

cursor.execute("SELECT * FROM students WHERE age > 19")
students_over_19 = cursor.fetchall()

connection.close()

students_over_19

Ошибка вставки: CHECK constraint failed: age >= 18
Ошибка вставки: UNIQUE constraint failed: students.email


[(1, 'Alice', 'alice@example.com', 20)]

### Задача 2025: Создание базовой ORM-модели и работа с данными (0 б)


1. Создайте базу данных `company.db` с таблицами:
   - `departments`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `name` (TEXT, UNIQUE, NOT NULL).
   - `employees`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `name` (TEXT, NOT NULL),
     - `salary` (INTEGER, CHECK(salary > 0)),
     - `department_id` (INTEGER, ForeignKey(`departments.id`)).
2. Добавьте 3 отдела:
   - `HR`, `IT`, `Sales`.
3. Добавьте по 2 сотрудника в каждый отдел.
4. Напишите запросы:
   - Вывести всех сотрудников с их отделами.
   - Увеличить зарплату сотрудников отдела `IT` на 10%.
   - Удалить сотрудников из отдела `Sales`.


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, CheckConstraint
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

Base = declarative_base()

# Модели
class Department(Base):
    __tablename__ = 'departments'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    employees = relationship("Employee", back_populates="department")

class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    salary = Column(Integer, CheckConstraint("salary > 0"), nullable=False)
    department_id = Column(Integer, ForeignKey('departments.id'))
    department = relationship("Department", back_populates="employees")

# Создание базы данных
engine = create_engine("sqlite:///company.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Добавление данных
if not session.query(Department).first():
    hr = Department(name="HR", employees=[Employee(name="Alice", salary=5000), Employee(name="Bob", salary=4500)])
    it = Department(name="IT", employees=[Employee(name="Charlie", salary=6000), Employee(name="David", salary=7000)])
    sales = Department(name="Sales", employees=[Employee(name="Eve", salary=4000), Employee(name="Frank", salary=3000)])
    session.add_all([hr, it, sales])
    session.commit()

# Запросы
# 1. Вывести всех сотрудников с отделами
employees = session.query(Employee).all()
print("Все сотрудники с отделами:")
for employee in employees:
    print(f"{employee.name} - {employee.department.name}, зарплата: {employee.salary}")

# 2. Увеличить зарплату сотрудников IT на 10%
session.query(Employee).filter(Employee.department.has(name="IT")).update(
    {Employee.salary: Employee.salary * 1.1}, synchronize_session="fetch"
)
session.commit()

# 3. Удалить сотрудников из отдела Sales
sales_department = session.query(Department).filter_by(name="Sales").first()
session.query(Employee).filter_by(department_id=sales_department.id).delete()
session.commit()

# Проверка оставшихся сотрудников
print("Сотрудники после изменений:")
for employee in session.query(Employee).all():
    print(f"{employee.name} - {employee.department.name}, зарплата: {employee.salary}")

session.close()

Все сотрудники с отделами:
Alice - HR, зарплата: 5000
Bob - HR, зарплата: 4500
Charlie - IT, зарплата: 6000
David - IT, зарплата: 7000
Eve - Sales, зарплата: 4000
Frank - Sales, зарплата: 3000
Сотрудники после изменений:
Alice - HR, зарплата: 5000
Bob - HR, зарплата: 4500
Charlie - IT, зарплата: 6600.000000000001
David - IT, зарплата: 7700.000000000001


### Задача 6: Оптимизация запросов с использованием `joinedload` из ORM (0.5 б)

1. Создайте SQLite-базу данных `social.db` с таблицами:
   - `users`: `id` (INTEGER, PRIMARY KEY), `name` (TEXT);
   - `posts`: `id` (INTEGER, PRIMARY KEY), `title` (TEXT), `user_id` (INTEGER).
2. Заполните таблицы следующими данными:
   - Пользователи: `Alice`, `Bob`.
   - Посты: для `Alice` — `Post 1`, `Post 2`; для `Bob` — `Post 3`.
3. Напишите код, который эффективно выводит пользователей и их посты с использованием `joinedload`.

https://docs.sqlalchemy.org/en/14/orm/loading_relationships.html#sqlalchemy.orm.joinedload

In [16]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship, joinedload

Base = declarative_base()

# Определение моделей
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    posts = relationship("Post", back_populates="user")

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="posts")

engine = create_engine("sqlite:///social.db", echo=False)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

if session.query(User).count() == 0:
    alice = User(name="Alice", posts=[Post(title="Post 1"), Post(title="Post 2")])
    bob = User(name="Bob", posts=[Post(title="Post 3")])
    session.add_all([alice, bob])
    session.commit()

users = session.query(User).options(joinedload(User.posts)).all()

for user in users:
    print(f"User: {user.name}")
    for post in user.posts:
        print(f"  Post: {post.title}")

session.close()

User: Alice
  Post: Post 1
  Post: Post 2
User: Bob
  Post: Post 3
